# Modelo de Posicionamiento en Mercado (0-2x)

Este notebook contiene todo el proceso: carga de datos, creación del target, entrenamiento con validación temporal, stacking con meta-modelo y guardado. Al final se incluye la sección de predicción y cálculo de métricas.

Objetivo: predecir la mejor posición diaria en el mercado (0 = cash, 1 = 100% mercado, 2 = 200% apalancado) usando retornos forward excess como señal.

## 1. Importaciones y configuración inicial

In [ ]:
import os
import warnings
import joblib
import lightgbm as lgb
import numpy as np
import pandas as pd
import polars as pl
from sklearn.linear_model import ElasticNetCV, RidgeCV
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")
np.random.seed(42)

## 2. Carga y preparación de datos

In [ ]:
train_path = "./train.csv"
assert os.path.exists(train_path), "No se encuentra train.csv"

train = pl.read_csv(train_path)
train = train.rename({"market_forward_excess_returns": "excess_return"})
train = train.with_columns(pl.all().cast(pl.Float64, strict=False))
train = train.drop_nulls(subset=["excess_return"])

df = train.to_pandas().set_index("date_id")

## 3. Creación del target óptimo [0, 2]

Transformamos el exceso de retorno futuro en una posición deseada:
- Multiplicamos por 40 (factor de escala aproximado para llevar señales a rango útil).
- Clip a [-1, 1] y luego convertimos a solo positivo [0, 2] (sin posiciones cortas).
- 0 = 100% cash, 1 = 100% mercado, 2 = 200% apalancado.

In [ ]:
# Factor 40 es aproximado, se puede ajustar según volatilidad histórica
df["target"] = np.clip(df["excess_return"] * 40, -1.0, 1.0)
df["target"] = df["target"].clip(lower=0) * 2.0  # rango [0, 2]

features = [c for c in df.columns if c not in ["date_id", "excess_return", "forward_returns", "risk_free_rate", "target"]]
X = df[features].fillna(0)
y = df["target"].values

print(f"Usando {len(features)} features | Target ahora es posición [0-2]")

## 4. Validación temporal y entrenamiento out-of-fold

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
oof_enet = np.zeros(len(X))
oof_lgb = np.zeros(len(X))

for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    scaler = StandardScaler()
    X_tr_s = scaler.fit_transform(X_tr)
    X_val_s = scaler.transform(X_val)
    
    # ElasticNet
    enet = ElasticNetCV(
        l1_ratio=[0.1, 0.5, 0.9, 1.0],
        alphas=np.logspace(-5, 1, 15),
        cv=3,
        max_iter=20000,
        n_jobs=-1,
        random_state=42,
    )
    enet.fit(X_tr_s, y_tr)
    oof_enet[val_idx] = np.clip(enet.predict(X_val_s), 0, 2)
    
    # LightGBM
    lgb_model = lgb.train(
        {
            "objective": "regression",
            "metric": "rmse",
            "learning_rate": 0.05,
            "num_leaves": 100,
            "max_depth": 10,
            "min_data_in_leaf": 30,
            "feature_fraction": 0.8,
            "bagging_fraction": 0.8,
            "bagging_freq": 5,
            "verbosity": -1,
            "seed": 42,
        },
        lgb.Dataset(X_tr, y_tr),
        num_boost_round=2500,
        valid_sets=[lgb.Dataset(X_val, y_val)],
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)],
    )
    oof_lgb[val_idx] = np.clip(lgb_model.predict(X_val), 0, 2)
    
    print(f"Fold {fold+1} - ENet: {root_mean_squared_error(y_val, oof_enet[val_idx]):.4f} | LGB: {root_mean_squared_error(y_val, oof_lgb[val_idx]):.4f}")

## 5. Meta-modelo (stacking)

In [ ]:
meta_X = np.column_stack([oof_enet, oof_lgb])
meta_model = RidgeCV(alphas=np.logspace(-3, 3, 13))
meta_model.fit(meta_X, y)

print(f"Meta weights -> ENet: {meta_model.coef_[0]:.3f}, LGB: {meta_model.coef_[1]:.3f}")

## 6. Entrenamiento final con todos los datos

In [ ]:
scaler_final = StandardScaler().fit(X)
X_scaled = scaler_final.transform(X)

final_enet = ElasticNetCV(
    l1_ratio=[0.1, 0.5, 0.9, 1.0],
    alphas=np.logspace(-5, 1, 20),
    cv=5,
    max_iter=50000,
    n_jobs=-1,
    random_state=42,
)
final_enet.fit(X_scaled, y)

final_lgb = lgb.train(
    {
        "objective": "regression",
        "learning_rate": 0.03,
        "num_leaves": 128,
        "max_depth": 10,
        "min_data_in_leaf": 30,
        "feature_fraction": 0.7,
        "bagging_fraction": 0.7,
        "bagging_freq": 5,
        "verbosity": -1,
        "seed": 42,
    },
    lgb.Dataset(X, y),
    num_boost_round=5000,
)

# Guardar modelo completo
joblib.dump(
    {
        "scaler": scaler_final,
        "enet": final_enet,
        "lgb": final_lgb,
        "meta": meta_model,
        "features": features,
    },
    "final_model.pkl",
)

print("Modelo final guardado como final_model.pkl")

## 7. Resumen de parámetros finales

In [ ]:
print("\n=== Resumen de entrenamiento final ===")
print(f"Features usadas: {len(features)}")
print(f"ElasticNet final:")
print(f" - Mejor alpha: {final_enet.alpha_:.6f}")
print(f" - Mejor l1_ratio: {final_enet.l1_ratio_}")
print(f" - Número de iteraciones: {final_enet.n_iter_}")

print("\nLightGBM final:")
params_lgb = final_lgb.params
print(f" - learning_rate: {params_lgb.get('learning_rate', 'N/A')}")
print(f" - num_leaves: {params_lgb.get('num_leaves', 'N/A')}")
print(f" - max_depth: {params_lgb.get('max_depth', 'N/A')}")
print(f" - min_data_in_leaf: {params_lgb.get('min_data_in_leaf', 'N/A')}")
print(f" - Número de boosting rounds: {final_lgb.current_iteration()}")

print("\nMeta modelo RidgeCV:")
print(f" - Mejor alpha: {meta_model.alpha_}")
print(f" - Coeficientes: ENet = {meta_model.coef_[0]:.4f}, LGB = {meta_model.coef_[1]:.4f}")

## 8. Predicción y métricas (sobre test y train)

In [ ]:
# Cargar modelo (si ya está entrenado)
model = joblib.load("final_model.pkl")
scaler = model["scaler"]
enet = model["enet"]
lgb = model["lgb"]
meta = model["meta"]
features = model["features"]

def prepare_X(df: pl.DataFrame) -> np.ndarray:
    missing = [f for f in features if f not in df.columns]
    if missing:
        df = df.with_columns([pl.lit(0.0).alias(f) for f in missing])
    X = df.select(features).fill_null(0).to_numpy().astype(np.float64)
    if np.isnan(X).any():
        X = np.nan_to_num(X, nan=0.0)
    return X

# Cargar datos una sola vez
test_df = pl.read_csv("test.csv")
train_df = pl.read_csv("train.csv")

# Predicción en test
X_test = prepare_X(test_df)
X_test_s = scaler.transform(X_test)
pred_enet = enet.predict(X_test_s)
pred_lgb = lgb.predict(X_test)
position = np.clip(meta.predict(np.column_stack([pred_enet, pred_lgb])), 0, 2)

submission = pd.DataFrame({
    "date_id": test_df["date_id"].to_list(),
    "position": position.flatten()
})
if "is_scored" in test_df.columns:
    submission = submission[test_df["is_scored"].to_numpy().astype(bool)].reset_index(drop=True)

submission.to_csv("submission.csv", index=False)
print(f"SUBMISSION → {len(submission)} días puntuables → submission.csv guardado")

# Métricas en train
X_train = prepare_X(train_df)
X_train_s = scaler.transform(X_train)
pred_enet_train = enet.predict(X_train_s)
pred_lgb_train = lgb.predict(X_train)
position_train = np.clip(meta.predict(np.column_stack([pred_enet_train, pred_lgb_train])), 0, 2)

excess = train_df["market_forward_excess_returns"].to_numpy()
strategy_ret = position_train.flatten() * excess
ann_ret = strategy_ret.mean() * 252
ann_vol = strategy_ret.std() * np.sqrt(252)
sharpe = ann_ret / ann_vol if ann_vol > 0 else 0
market_vol = excess.std() * np.sqrt(252)
vol_ratio = ann_vol / (1.2 * market_vol)
penalty = max(vol_ratio - 1, 0) ** 2
adj_sharpe = sharpe * (1 - penalty)
mae = np.mean(np.abs(strategy_ret - excess))
rmse = np.sqrt(np.mean((strategy_ret - excess)**2))

print("\n" + "="*60)
print(" RESULTADOS FINALES - TU ESTRATEGIA (STACKING)")
print("="*60)
print(f"Retorno anualizado : {ann_ret*100:6.2f}%")
print(f"Volatilidad anualizada : {ann_vol*100:6.2f}%")
print(f"Sharpe Ratio : {sharpe:.4f}")
print(f"Sharpe Ajustado : {adj_sharpe:.4f}")
print(f"MAE vs Benchmark : {mae:.6f}")
print(f"RMSE vs Benchmark : {rmse:.6f}")
print(f"Vol vs límite 120% : {vol_ratio:.3f}x → penalización {penalty:.1%}")
print(f"Días evaluados : {len(excess)}")
print("="*60)